In [1]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german

from aif360.algorithms.inprocessing.adversarial_debiasing_dnn5 import AdversarialDebiasingDnn5

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
print('hiii')
# Get the dataset and split into train and test
dataset_orig = load_preproc_data_adult()

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.8], shuffle=True)
# print out some labels, names, etc.
sess = tf.Session()
plain_model = AdversarialDebiasingDnn5(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

hiii


### Learn plan classifier without debiasing

In [2]:
dataset_nodebiasing_train_, dataset_nodebiasing_test_ = plain_model.fit_and_pred(dataset_orig_train, dataset_orig_test)




Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





epoch 0; iter: 0; batch classifier loss: 0.687487
epoch 0; iter: 200; batch classifier loss: 0.389736
epoch 1; iter: 0; batch classifier loss: 0.439247
epoch 1; iter: 200; batch classifier loss: 0.320585
epoch 2; iter: 0; batch classifier loss: 0.447998
epoch 2; iter: 200; batch classifier loss: 0.418517
epoch 3; iter: 0; batch classifier loss: 0.478368
epoch 3; iter: 200; batch classifier loss: 0.437139
epoch 4; iter: 0; batch classifier loss: 0.404894
epoch 4; iter: 200; batch classifier loss: 0.419438
epoch 5; iter: 0; batch classifier loss: 0.372289
epoch 5; iter: 200; batch classifier loss: 0.336985
epoch 6; iter: 0; batch classifier loss: 0.404709
epoch 6; iter: 200; batch classifier loss: 0.417492
epoch 7; iter: 0; batch classifier loss: 0.369820
epoch 7; iter: 200; batch classifier loss: 0.418875
epoch 8; 

In [2]:
# Apply the plain model to test data
dataset_nodebiasing_train = plain_model.predict_with_load_gra(dataset_orig_train)
dataset_nodebiasing_test = plain_model.predict_with_load_gra(dataset_orig_test)
print('ok')




Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






INFO:tensorflow:Restoring parameters from ../org-model-merge/adult/9/test.model
INFO:tensorflow:Restoring parameters from ../org-model-merge/adult/9/test.model
ok


In [3]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.000000
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.000000


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.238612
Test set: Balanced classification accuracy = 0.500000
Test set: Disparate impact = 1.000000
Test set: Equal opportunity difference = 0.000000
Test set: Average odds difference = 0.000000
Test set: Theil_index = 0.033146


### Apply in-processing algorithm based on adversarial learning

In [4]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [5]:
# Learn parameters with debias set to True
debiased_model = AdversarialDebiasingDnn5(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)

In [6]:
dataset_debiasing_train_, dataset_debiasing_test_ = debiased_model.fit_and_pred(dataset_orig_train, dataset_orig_test)


epoch 0; iter: 0; batch classifier loss: 0.680085; batch adversarial loss: 0.685441
epoch 0; iter: 200; batch classifier loss: 0.464239; batch adversarial loss: 0.634047
epoch 1; iter: 0; batch classifier loss: 0.479699; batch adversarial loss: 0.642877
epoch 1; iter: 200; batch classifier loss: 0.437703; batch adversarial loss: 0.631411
epoch 2; iter: 0; batch classifier loss: 0.382678; batch adversarial loss: 0.646070
epoch 2; iter: 200; batch classifier loss: 0.429821; batch adversarial loss: 0.593361
epoch 3; iter: 0; batch classifier loss: 0.405579; batch adversarial loss: 0.645083
epoch 3; iter: 200; batch classifier loss: 0.520919; batch adversarial loss: 0.692524
epoch 4; iter: 0; batch classifier loss: 0.446790; batch adversarial loss: 0.645860
epoch 4; iter: 200; batch classifier loss: 0.457840; batch adversarial loss: 0.595926
epoch 5; iter: 0; batch classifier loss: 0.423679; batch adversarial loss: 0.627245
epoch 5; iter: 200; batch classifier loss: 0.566020; batch advers

In [ ]:
# Apply the plain model to test data
dataset_debiasing_train = debiased_model.predict_with_load_gra(dataset_orig_train)
dataset_debiasing_test = debiased_model.predict_with_load_gra(dataset_orig_test)

ERROR! Session/line number was not unique in database. History logging moved to new session 121


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\Software\anaconda3\envs\newtf115\lib\site-packages\IPython\core\interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\wpkks\AppData\Local\Temp\ipykernel_29516\2917289970.py", line 2, in <module>
    dataset_debiasing_train = debiased_model.predict_with_load_gra(dataset_orig_train)
  File "D:\PycharmProjects\AIF360\aif360\algorithms\transformer.py", line 27, in wrapper
    new_dataset = func(self, *args, **kwargs)
  File "D:\PycharmProjects\AIF360\aif360\algorithms\inprocessing\adversarial_debiasing_dnn5.py", line 513, in predict_with_load_gra
    self.fit_without_train(dataset)
  File "D:\PycharmProjects\AIF360\aif360\algorithms\transformer.py", line 27, in wrapper
    new_dataset = func(self, *args, **kwargs)
  File "D:\PycharmProjects\AIF360\aif360\algorithms\inprocessing\adversarial_debiasing_dnn5.py", line 298, in fit_without_train
    pred_protected_attributes_labels, pred_p

In [ ]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Plain model - without debiasing - classification metrics"))
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.000000
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.000000


#### Model - with debiasing - dataset metrics

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 120
Traceback (most recent call last):
  File "D:\Software\anaconda3\envs\newtf115\lib\site-packages\IPython\core\interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\wpkks\AppData\Local\Temp\ipykernel_29516\3334198007.py", line 8, in <module>
    metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train,
NameError: name 'dataset_debiasing_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Software\anaconda3\envs\newtf115\lib\site-packages\IPython\core\interactiveshell.py", line 2098, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\S